In [19]:

#import libraries necessary for organizing and cleaning our data:
import numpy as np
import pandas as pd
import math
import re
import datetime
data = pd.read_csv('Kickstarter.csv',encoding = "ISO-8859-1")

In [ ]:
#SETTING UP DATA FOR ANALYSIS

data = pd.read_csv('Kickstarter.csv',index_col=0)
import math
import re
import datetime
#put date in line
#for i in range(len(data)-1):
    #data['Launch_Date'][i] = data['Launch_Date'][i+1]
for i in range(len(data)-2):
    data['Launch_Date'][i] = data['Launch_Date'][i+2]

#delete extra lines generated by excel
#data = data.iloc[::2]
data = data.iloc[:int(len(data)/2)]

# multiple-use length of data set
n = len(data)

# convert string of previously created projects into number of created projects:

for i in range(n):
    try:
        if np.isnan(data['Nb_Created_Projects'][i])==True:
            data['Nb_Created_Projects'][i] = 0
            
        elif data['Nb_Created_Projects'][i][0] == 'F':
            data['Nb_Created_Projects'][i] = 1
        
        else:
            data['Nb_Created_Projects'][i] = data['Nb_Created_Projects'][i][0]
    except:
        if data['Nb_Created_Projects'][i][0] == 'F':
            data['Nb_Created_Projects'][i] = 1
        else:
            data['Nb_Created_Projects'][i] = data['Nb_Created_Projects'][i][0]
    

# convert "We Love Tag" into numeric:
for i in range(n):
    if data['We_Love_Tag'][i] == 'Project We Love':
        data['We_Love_Tag'][i] = 1
    else:
        data['We_Love_Tag'][i] = 0

# convert location into seperate strings to be able to analyze at a state vs city vs country level:
data['City'] = 'a'
data['State'] = 'a'
for i in range(n):
    data['Location'][i] = data['Location'][i].split(',')
    data['Location'][i][0] = data['Location'][i][0].replace(" ", "")
    data['Location'][i][1] = data['Location'][i][1].replace(" ", "")
    data['City'][i] = data['Location'][i][0] 
    data['State'][i] = data['Location'][i][1]


# configurating launch date of project:   
for i in range(n):
    data['Launch_Date'][i] = re.findall(r"[\w']+",(data['Launch_Date'][i]))
    
    #convert months into number
months = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
for i in range(n):
    data['Launch_Date'][i][0] = months[data['Launch_Date'][i][0]]

    #convert numbers into date
for i in range(n):
    data['Launch_Date'][i] = datetime.date(int(data['Launch_Date'][i][2]),int(data['Launch_Date'][i][0]),int(data['Launch_Date'][i][1]))

#configurating end date of the project:
for i in range(n):
    data['End_Date'][i] = data['End_Date'][i].split(',')

for i in range(n):
    data['End_Date'][i][0] = months[data['End_Date'][i][0]]

for i in range(n):
    data['End_Date'][i] = datetime.date(int(data['End_Date'][i][2]),int(data['End_Date'][i][0]),int(data['End_Date'][i][1]))

data['Project_Total_Days'] = 1
for i in range(n):
    data['Project_Total_Days'][i] = (data['End_Date'][i] - data['Launch_Date'][i]).days    

#number of words in description:
data['Words_In_Desc'] = 1
for i in range(n):
    data['Words_In_Desc'][i] = len(data['Description'][i].split(' '))

#number of words in title:
data['Words_In_Title'] = 1
for i in range(n):
    data['Words_In_Title'][i] = len(data['Title'][i].split(' '))


#defining sub_type from type 
    #setting up sub-type dictionnary
data1 = pd.read_csv('Dict_for_categories.csv')
dict_types = {}
for i in data1:
    dict_types.update({i:list(data1[i][0:20])})

data['Sub_Type'] = 'a'

#creating the sub-type:

for i in range(n):
    data['Sub_Type'][i] = data['Type'][i]
    for a in dict_types:
        if data['Type'][i] in dict_types[a]:
            data['Type'][i] = dict_types[a][0]
            
#% of a project that has been funded:
data['Funded_Perc'] = 1
for i in range(n):
    data['Funded_Perc'][i] = float((data['Pledged_Dollars'][i]/data['Pledged_Goal'][i])*100)

#% of the time remianing for project
data['Days_Left_Percentage'] = 1
for i in range(n):
    if (data['End_Date'][i] - datetime.date(2018,10,16)).days > 0:
        data['Days_Left_Percentage'][i] = 100*((data['End_Date'][i] - datetime.date(2018,10,16)).days)/data['Project_Total_Days'][i]    
    else:
        data['Days_Left_Percentage'][i] = 0